In [3]:
# ライブラリのインポート
from bokeh import palettes
from bokeh.transform import transform, log_cmap
from bokeh.layouts import row, column
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import DatetimeTickFormatter, Legend, ColorBar, LinearColorMapper, ColumnDataSource
import pandas as pd
import numpy as np

# 折れ線グラフ
連続した数値の変化を表す定番グラフ。ただし、ラベルの種類数が10を超えたあたりから線が重なるなどで見づらくなる。<br>
推奨数は5で、それ以上の場合はグラフを複数に分けるほうが無難<br>
データはhttps://gist.github.com/mbostock/3884955 からのもので、ある時期の各地域の温度（°F）の関係図

In [4]:
df = pd.read_csv(
    "../data/plot.csv",
    index_col="date",
    parse_dates=True,
    date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d"))  # yyyymmdd -> yyyy-mm-ddに変換

df.head()

C:\Users\atsu9\AppData\Local\Temp\ipykernel_21260\1058361137.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


,New York,San Francisco,Austin
date,,,
2011-10-01,63.4,62.7,72.2
2011-10-02,58.0,59.9,67.7
2011-10-03,53.3,59.1,69.4
2011-10-04,55.7,58.8,68.0
2011-10-05,64.2,58.7,72.4


In [5]:
output_notebook()

Loading BokehJS ...

In [19]:
p = figure(
    width=700,
    height=400,
    title="City Temperature",
    x_axis_type = "datetime")

colors = palettes.Category10[3]

x_format = "%Y/%m/%d"

for i, city_name in enumerate(df.columns):
    p.line(
        x = df.index.to_numpy(),
        y = df.loc[:, city_name],
        line_color=colors[i],
        legend_label = city_name
        )

p.xaxis.formatter = DatetimeTickFormatter(
    days=x_format,
    months=x_format,
    years=x_format
    )
p.xaxis.major_label_orientation = np.pi/2
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "Temeperature[F°]"
p.add_layout(p.legend[0], "right")

show(p)

IndexError: tuple index out of range

In [7]:
colors = palettes.Category10[3]
x_format = "%Y/%m"
p_list = []

for i, city_name in enumerate(df.columns):
    p = figure(
        width=700,
        height=300,
        x_axis_type = "datetime",
        title=city_name
        )
    p.line(
        x = df.index.to_numpy(),
        y = df.loc[:, city_name],
        line_color=colors[i]
        )
    p.xaxis.formatter = DatetimeTickFormatter(
        months=x_format,
        years=x_format
    )
    p.xaxis.axis_label = "date"
    p.yaxis.axis_label = "Temperature[F°]"
    p_list.append(p)


show(column(p_list))

# 散布図
2つの数値を座標として捉えたときのデータの偏りを調べる方法。1つの系列が自国の場合はX軸を時刻に割り当てる場合が多い。3つめの数値があるときは点の代償などで大小を表現することができる。ただし、見る人が対象となるラベルを見つけなければならないため、折れ線グラフや時系列ヒートマップのように時間的な数値の増減を表現するのは苦手。あくまで、「どこに偏りがあるか？」を見るのに使う。<br>
データはボストンの住宅価格データセットを用いた。
- CRIM： 町別の「犯罪率」
- ZN： 25,000平方フィートを超える区画に分類される住宅地の割合＝「広い家の割合」
- INDUS： 町別の「非小売業の割合」
- CHAS： チャールズ川のダミー変数（区画が川に接している場合は1、そうでない場合は0）＝「川の隣か」
- NOX： 「NOx濃度（0.1ppm単位）」＝一酸化窒素濃度（parts per 10 million単位）。この項目を目的変数とする場合もある
- RM： 1戸当たりの「平均部屋数」
- AGE： 1940年より前に建てられた持ち家の割合＝「古い家の割合」
- DIS： 5つあるボストン雇用センターまでの加重距離＝「主要施設への距離」
- RAD： 「主要高速道路へのアクセス性」の指数
- TAX： 10,000ドル当たりの「固定資産税率」
- PTRATIO： 町別の「生徒と先生の比率」
- B： 「1000(Bk - 0.63)」の二乗値。Bk＝「町ごとの黒人の割合」を指す
- LSTAT： 「低所得者人口の割合」
- MEDV：「住宅価格」（1000ドル単位）の中央値。通常はこの数値が目的変数として使われる

In [8]:
scatter_df = pd.read_csv('../data/scatter.csv')
# その地域の住宅の平均部屋数と低所得者数から住宅価格との関連性を確認する
scatter_df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [9]:
p = figure(
    width=700,
    height=400,
    title="Boston House Price")

src = ColumnDataSource(dict(
    x=scatter_df.loc[:, "rm"],
    y=scatter_df.loc[:, "lstat"],
    c=scatter_df.loc[:, "medv"],
    s=scatter_df.loc[:, "medv"]**0.6
    ))

color_range = 100
target = "medv"
colors = palettes.inferno(color_range)[::-1]

color_mapper = LinearColorMapper(
    palette=colors,
    # low=df.loc[:, target].min(),
    low=0,
    high=scatter_df.loc[:, target].max())

color_bar = ColorBar(
    color_mapper=color_mapper,
    width=550,
    height=20,
    location=(0, 0),
    orientation="horizontal",
    bar_line_color="black",
    major_tick_line_color="black")

p.scatter(
    x="x", y="y", size = "s",
    marker="circle",
    fill_color = transform("c", color_mapper),
    source = src)

p.xaxis.axis_label = "average number of rooms per dweling"
p.yaxis.axis_label = "Tlower status of the population"
p.add_layout(color_bar, "below")

show(p)

# ヒートマップ
数値2つを座標として、3つ目の数値をその座標の色を決定するために使うことで、全体の数値のバラつきや、高い・低い値の集中しているエリアを知ることができる。数値の代わりに時刻にすることもできなくはないが、あまりそういうグラフは見たことがない。<br>
データはhttps://github.com/mwaskom/seaborn-../data/blob/master/flights.csv からのもので、各年の各月における飛行機の乗客数

# 積み重ね棒グラフ/集合棒グラフ
2つのラベルと1つの数値があったときに、これらの数値の大小を比較する方法。2つ以上のラベルをうまく表現できる数少ない方法だと思うが、データ数（≒2つのラベルの組み合わせ数）が50を超えたぐらいから図が煩雑になって読み取りが難しくなってしまう。<br>
データはhttps://www.enecho.meti.go.jp/about/whitepaper/2014html/2-1-1.html からのもので、日本の一次エネルギー国内供給の推移である。

In [10]:
df = pd.read_csv('../data/stacked_bar_chart.csv')
df.head()

,年度,石油,石炭,天然ガス,原子力,水力,新エネルギー・地熱等
0,1965,3.56,1.87,0.08,0.00,0.75,0.11
1,1966,4.13,1.92,0.09,0.01,0.77,0.11
2,1967,5.11,2.29,0.09,0.01,0.67,0.11
3,1968,5.95,2.42,0.10,0.01,0.72,0.12
4,1969,7.19,2.59,0.12,0.01,0.73,0.13


In [11]:
column_list = [column for column in df.columns[1:]]
src = ColumnDataSource(df)

p = figure(
    width=1200,
    height=400,
    title="Domestic Energy Supplies")

p.vbar_stack(
    stackers=column_list,
    x="年度",
    width=0.6,
    color=palettes.RdYlGn[len(column_list)],
    source=src,
    legend_label=column_list)
p.x_range.start = df["年度"].min() - 1
p.x_range.end = df["年度"].max() + 1
p.xgrid.grid_line_color = None

p.add_layout(p.legend[0], place="right")


show(p)

# 棒グラフ

データは東京の気温情報https://smart-hint.com/tokyo-temperature/ から

In [12]:
df = pd.read_csv(
    '../data/bar.csv',
    index_col=0,
    parse_dates=True,
    date_parser=lambda x: pd.to_datetime(x, format='%Y/%m/%d') # 文字列から時間にする
    )
df.head()

C:\Users\atsu9\AppData\Local\Temp\ipykernel_21260\2208147134.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


,平均気温,最高気温,最低気温,月
日付,,,,
2021-01-01,4.4,10.5,-1.3,1
2021-01-02,4.8,10.8,0.1,1
2021-01-03,3.7,8.4,-0.4,1
2021-01-04,5.8,11.4,1.3,1
2021-01-05,6.0,10.0,2.5,1


In [13]:
mean_datas = [df[column].resample("M").mean() for column in df.columns[:-1]]
std_datas = [df[column].resample("M").std() for column in df.columns[:-1]]
dates = list({f"{i.year}{i.month:02}" for i in df.index.unique()})
upper = [temp + e for temp, e in zip(mean_datas, std_datas)]
lower = [temp - e for temp, e in zip(mean_datas, std_datas)]
# dates = list(dates)

In [14]:
data = {
    "年月":dates,
    "平均気温":mean_datas[0],
    "最高気温":mean_datas[1],
    "最低気温":mean_datas[2]}

source = ColumnDataSource(data)

In [15]:
from bokeh.transform import dodge
import datetime
from bokeh.models import Whisker

color_palette = palettes.Set1[3]

p = figure(x_range=dates, width=800, height=500)

for i, temp_type in enumerate(["平均気温", "最高気温", "最低気温"]):
    p.vbar(
        x=dodge(field_name="年月", value=-0.25+0.25*i, range=p.x_range),
        top=temp_type,
        source=source,
        width=0.2,
        color=color_palette[i],
        legend_label=temp_type)

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "年月"
p.yaxis.axis_label = "温度[°C]"
# p.legend.orientation = "horizontal"
p.add_layout(p.legend[0], place="right")

show(p)

# 箱ヒゲ図
データの分散をあらわすための可視化。図中では線の両端が最小値・最大値。箱の左側が第1四分位点、赤と緑の間の線が中央値、右側が第3四分位点となる。これによってラベル毎に数値のばらつきがどの程度あるか？ということを概ね理解することができる。この箱の部分が狭いほどその範囲に数値が集中していることを意味し、箱が広いほど数値がばらついていると読み取ることができる。たまに平均値を算出して棒グラフで表現するグラフを見かけるが、それだと数値が一部に集中しているのか、それともばらけているのか、ということまで分からないため、データの性質を見るという目的なら箱ひげ図のほうが望ましい。<br>
データはhttps://www.geeksforgeeks.org/box-plot-visualization-with-pandas-and-seaborn/を使用

In [16]:
df = pd.read_csv('../data/box_plot.csv')
display(df.head(3))
qs = df.groupby(["day"])["total_bill"].quantile([0.25, 0.5, 0.75])
display(qs)
qs = qs.unstack().reset_index()
qs.columns = ["day", "q1", "q2", "q3"]
display(qs.head())
df = pd.merge(df, qs, on="day", how="left")
iqr = df["q3"] - df["q1"]

df["upper"] = df["q3"] + 1.5*iqr
df["lower"] = df["q1"] - 1.5*iqr
days = df["day"].unique()
for day in days:
    idx = df[df["day"]==day].index
    if df.loc[idx, "total_bill"].max() < df.loc[idx, "upper"].max():
        df.loc[idx, "upper"] = df.loc[idx, "total_bill"].max()
    if df.loc[idx, "total_bill"].min() > df.loc[idx, "lower"].min():
        df.loc[idx, "lower"] = df.loc[idx, "total_bill"].min()

display(df.head(3))


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


day       
Fri   0.25    12.0950
      0.50    15.3800
      0.75    21.7500
Sat   0.25    13.9050
      0.50    18.2400
      0.75    24.7400
Sun   0.25    14.9875
      0.50    19.6300
      0.75    25.5975
Thur  0.25    12.4425
      0.50    16.2000
      0.75    20.1550
Name: total_bill, dtype: float64

,day,q1,q2,q3
0,Fri,12.0950,15.38,21.7500
1,Sat,13.9050,18.24,24.7400
2,Sun,14.9875,19.63,25.5975
3,Thur,12.4425,16.20,20.1550


,total_bill,tip,sex,smoker,day,time,size,q1,q2,q3,upper,lower
0,16.99,1.01,Female,No,Sun,Dinner,2,14.9875,19.63,25.5975,41.5125,7.25
1,10.34,1.66,Male,No,Sun,Dinner,3,14.9875,19.63,25.5975,41.5125,7.25
2,21.01,3.50,Male,No,Sun,Dinner,3,14.9875,19.63,25.5975,41.5125,7.25


In [17]:
source = ColumnDataSource(df)

p = figure(
    x_range=df["day"].unique(),
    y_range=[0, 60],
    title="bill and sex",
    background_fill_color="white")

whisker = Whisker(
    base="day",
    upper="upper",
    lower="lower",
    source=source)

p.add_layout(whisker)

p.vbar("day", 0.7, "q2", "q3", source=source)
p.vbar("day", 0.7, "q1", "q2", source=source)

show(p)

# ヒストグラム
箱ひげ図と同じく、どのようにデータが分散しているかを表現する可視化手法。箱ひげ図では中央値や四分位点がどこにあるかを示すだけだったが、どのヒストグラムではどの数値帯にデータが何件（または全体データ数の何%）が出現したかを表現するため、データのばらつきが克明にわかる。基本的に1系列の数値データのみしか扱えないので複数のラベルについて調べたいという場合はラベル毎に別々にグラフを作成する必要があるが、数値データの性質を調べるためには強力な可視化手法。各数値帯を棒グラフ状に表す方法の他に折れ線グラフを使う方法、または累積度数分布として表現する方法などもある。
(ラベル, 数値)のようなデータ形式に対して、複数のラベルのヒストグラムを同じグラフに描画することもできるが、粒度の粗いデータだと線の重なりが起きたときなどに調整が必要なため、素直に1系列でグラフ化するのがお勧めである。
データはhttps://raw.githubusercontent.com/selva86/datasets/master/diamonds.csv

In [18]:
df = pd.read_csv('../data/histgram.csv')
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
